In [1]:
import os, gc, cv2, torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from PIL import Image
from tqdm import tqdm
from transformers import AutoImageProcessor, ZoeDepthForDepthEstimation
import shutil

# ========== 模型加载 ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "Intel/zoedepth-nyu-kitti"
processor = AutoImageProcessor.from_pretrained(model_name)
model = ZoeDepthForDepthEstimation.from_pretrained(model_name).to(device)

# ========== 路径配置 ==========
base_dir = "/kaggle/input/kitti-dataset/depth_selection/val_selection_cropped"
img_dir = os.path.join(base_dir, "image")
gt_dir = os.path.join(base_dir, "groundtruth_depth")
K_dir = os.path.join(base_dir, "intrinsics")

save_dir = "/kaggle/working/"
os.makedirs(save_dir, exist_ok=True)
vis_dir = os.path.join(save_dir, "visualizations")
os.makedirs(vis_dir, exist_ok=True)
pred_dir = os.path.join(save_dir, "predicted_depth")
os.makedirs(pred_dir, exist_ok=True)

# ========== 可视化：增加“原始预测 + 对齐预测”并排展示 ==========
def save_vis(rgb, gt, pred_metric, pred_aligned, mask, save_path):
    plt.figure(figsize=(20, 5))
    valid_all = mask & (gt > 0) & np.isfinite(gt)
    valid_metric = valid_all & np.isfinite(pred_metric)
    valid_aligned = valid_all & np.isfinite(pred_aligned)

    # 统一色标范围（用 GT 与两种预测的共同分位数）
    all_depths = []
    if np.any(valid_metric):
        all_depths.append(gt[valid_metric]); all_depths.append(pred_metric[valid_metric])
    if np.any(valid_aligned):
        all_depths.append(gt[valid_aligned]); all_depths.append(pred_aligned[valid_aligned])
    if len(all_depths) == 0:  # 兜底
        all_depths = [gt[valid_all]] if np.any(valid_all) else [gt.flatten()]
    all_depths = np.concatenate(all_depths)
    vmin = np.percentile(all_depths, 2)
    vmax = np.percentile(all_depths, 98)

    plt.subplot(1, 4, 1)
    plt.imshow(rgb)
    plt.title("RGB")
    plt.axis("off")

    plt.subplot(1, 4, 2)
    plt.imshow(gt, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("GT Depth")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.subplot(1, 4, 3)
    plt.imshow(pred_metric, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("Predicted (Metric)")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.subplot(1, 4, 4)
    plt.imshow(pred_aligned, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("Predicted (Aligned)")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()

# ========== 评估函数 ==========
def compute_abs_rel(pred, gt, mask):
    valid = mask & (gt > 0) & np.isfinite(gt) & np.isfinite(pred)
    pred, gt = pred[valid], gt[valid]
    if len(gt) == 0:
        return np.nan
    return np.mean(np.abs(pred - gt) / gt)

def compute_delta1(pred, gt, mask):
    valid = mask & (gt > 1e-6) & (pred > 1e-6) & np.isfinite(gt) & np.isfinite(pred)
    pred, gt = pred[valid], gt[valid]
    if len(gt) == 0:
        return np.nan
    thresh = np.maximum(pred / gt, gt / pred)
    return np.mean(thresh < 1.25) if len(thresh) > 0 else np.nan

def scale_match(pred, gt, mask):
    valid = mask & np.isfinite(gt) & np.isfinite(pred)
    pred_v, gt_v = pred[valid], gt[valid]
    denom = np.sum(pred_v ** 2)
    if denom <= 0:
        return 1.0
    return float(np.sum(gt_v * pred_v) / denom)

# ========== 主处理流程 ==========
results = []
img_files = sorted([f for f in os.listdir(img_dir) if f.endswith(".png")])

for img_file in tqdm(img_files, desc="Evaluating"):
    try:
        # 构建文件路径
        rgb_path = os.path.join(img_dir, img_file)
        gt_file = re.sub("image", "groundtruth_depth", img_file, count=1)
        gt_path = os.path.join(gt_dir, gt_file)
        K_path = os.path.join(K_dir, img_file.replace(".png", ".txt"))

        # 加载 RGB
        rgb = cv2.imread(rgb_path)
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        input_image = Image.fromarray(rgb)

        # 加载 GT（PNG 转 float, KITTI 以毫米存储）
        gt_png = cv2.imread(gt_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
        gt_depth = gt_png / 256.0  # 转为米
        mask = gt_depth > 0

        # （可选）加载相机内参
        try:
            with open(K_path, 'r') as f:
                vals = list(map(float, f.read().split()))
                fx, fy = vals[0], vals[4]
                cx, cy = vals[2], vals[5]
            K = torch.tensor([[fx, 0, cx],
                              [0, fy, cy],
                              [0, 0, 1]], dtype=torch.float32).unsqueeze(0).to(device)
        except Exception:
            K = None

        # ZoeDepth 推理
        inputs = processor(images=input_image, return_tensors="pt").to(device)
        # 若需要也可尝试传入相机内参（某些权重不需要）
        # if K is not None:
        #     inputs["camera_intrinsics"] = K

        with torch.no_grad():
            outputs = model(**inputs)

        processed = processor.post_process_depth_estimation(
            outputs, source_sizes=[(rgb.shape[0], rgb.shape[1])]
        )
        pred_depth = processed[0]["predicted_depth"].squeeze().cpu().numpy()  # 原始预测 = Metric

        # 尺寸对齐
        if pred_depth.shape != gt_depth.shape:
            pred_depth = cv2.resize(pred_depth, (gt_depth.shape[1], gt_depth.shape[0]),
                                    interpolation=cv2.INTER_LINEAR)

        # ===== Metric 评测（原始预测，不做缩放对齐） =====
        absrel_metric = compute_abs_rel(pred_depth, gt_depth, mask)
        delta1_metric = compute_delta1(pred_depth, gt_depth, mask)

        # ===== Relative 评测（scale 对齐后）=====
        scale = scale_match(pred_depth, gt_depth, mask)
        pred_aligned = pred_depth * scale
        absrel_rel = compute_abs_rel(pred_aligned, gt_depth, mask)
        delta1_rel = compute_delta1(pred_aligned, gt_depth, mask)

        results.append([img_file, absrel_metric, delta1_metric, absrel_rel, delta1_rel])

        print(f"[{img_file}] "
              f"Metric -> AbsRel: {absrel_metric:.4f}, δ1: {delta1_metric * 100:.2f}% | "
              f"Relative -> AbsRel: {absrel_rel:.4f}, δ1: {delta1_rel * 100:.2f}%")

        # 保存可视化（含 Metric & Aligned）
        vis_path = os.path.join(vis_dir, img_file.replace(".png", "_vis.png"))
        save_vis(rgb, gt_depth, pred_depth, pred_aligned, mask, vis_path)

        # 保存对齐后的预测深度（如需也可另存原始预测）
        np.save(os.path.join(pred_dir, img_file.replace(".png", "_pred_metric.npy")), pred_depth)
        np.save(os.path.join(pred_dir, img_file.replace(".png", "_pred_aligned.npy")), pred_aligned)

        # 内存清理
        del rgb, gt_depth, pred_depth, pred_aligned, inputs, outputs
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    except Exception as e:
        print(f"跳过 {img_file}，错误：{e}")
        continue

# ========== 保存评估结果 ==========
df = pd.DataFrame(results, columns=[
    "image",
    "AbsRel_metric", "Delta1_metric",
    "AbsRel_relative", "Delta1_relative"
])

# 追加均值
mean_row = pd.DataFrame([[
    "mean",
    df["AbsRel_metric"].mean(),
    df["Delta1_metric"].mean(),
    df["AbsRel_relative"].mean(),
    df["Delta1_relative"].mean()
]], columns=df.columns)

df = pd.concat([df, mean_row], ignore_index=True)
csv_path = os.path.join(save_dir, "eval_results.csv")
df.to_csv(csv_path, index=False)
print(f"\n所有评估结果保存至：{csv_path}")

# ========== 打包 ==========
zip_pred_path = os.path.join(save_dir, "predicted_depth.zip")
shutil.make_archive(base_name=zip_pred_path.replace('.zip', ''), format='zip', root_dir=pred_dir)
print(f"\n预测深度npy已打包至: {zip_pred_path}")

zip_vis_path = os.path.join(save_dir, "visualizations.zip")
shutil.make_archive(base_name=zip_vis_path.replace('.zip', ''), format='zip', root_dir=vis_dir)
print(f"打包完成: {zip_vis_path}")


2025-08-11 22:27:37.646030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754951257.834335      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754951257.890558      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

...

[2011_10_03_drive_0047_sync_image_0000000785_image_02.png] Metric -> AbsRel: 0.2589, δ1: 33.52% | Relative -> AbsRel: 0.1002, δ1: 98.07%


Evaluating: 100%|█████████▉| 999/1000 [1:18:05<00:07,  7.50s/it]

[2011_10_03_drive_0047_sync_image_0000000791_image_03.png] Metric -> AbsRel: 0.2458, δ1: 39.50% | Relative -> AbsRel: 0.1103, δ1: 97.39%


Evaluating: 100%|██████████| 1000/1000 [1:18:12<00:00,  4.69s/it]



所有评估结果保存至：/kaggle/working/eval_results.csv

预测深度npy已打包至: /kaggle/working/predicted_depth.zip
打包完成: /kaggle/working/visualizations.zip
